## Running A New Logistic Regression Model

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import sklearn

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import logit
import scipy.stats as stats

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import export_graphviz, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score



import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.4f}'.format)


random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Exploratory Data Analysis

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,insurance,fh,smoking,dm
0,1000,203.0000,82,56.0000,3.6000,4.3100,Buckingham,46,female,62.0000,121.0000,medium,118.0000,59.0000,nan,nan,29.0000,38.0000,720.0000,1,0,3,no
1,1001,165.0000,97,24.0000,6.9000,4.4400,Buckingham,29,female,64.0000,218.0000,large,112.0000,68.0000,nan,nan,46.0000,48.0000,360.0000,0,0,2,no
2,1002,228.0000,92,37.0000,6.2000,4.6400,Buckingham,58,female,61.0000,256.0000,large,190.0000,92.0000,185.0000,92.0000,49.0000,57.0000,180.0000,2,0,2,no
3,1003,78.0000,93,12.0000,6.5000,4.6300,Buckingham,67,male,67.0000,119.0000,large,110.0000,50.0000,nan,nan,33.0000,38.0000,480.0000,1,0,3,no
4,1005,249.0000,90,28.0000,8.9000,7.7200,Buckingham,64,male,68.0000,183.0000,medium,138.0000,80.0000,nan,nan,44.0000,41.0000,300.0000,0,0,3,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,41506,296.0000,369,46.0000,6.4000,16.1100,Louisa,53,male,69.0000,173.0000,medium,138.0000,94.0000,130.0000,94.0000,35.0000,39.0000,210.0000,2,0,1,yes
399,41507,284.0000,89,54.0000,5.3000,4.3900,Louisa,51,female,63.0000,154.0000,medium,140.0000,100.0000,146.0000,102.0000,32.0000,43.0000,180.0000,2,0,3,no
400,41510,194.0000,269,38.0000,5.1000,13.6300,Louisa,29,female,69.0000,167.0000,small,120.0000,70.0000,nan,nan,33.0000,40.0000,20.0000,0,1,2,yes
401,41752,199.0000,76,52.0000,3.8000,4.4900,Louisa,41,female,63.0000,197.0000,medium,120.0000,78.0000,nan,nan,41.0000,48.0000,255.0000,2,0,2,no


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         403 non-null    int64  
 1   chol       402 non-null    float64
 2   stab.glu   403 non-null    int64  
 3   hdl        402 non-null    float64
 4   ratio      402 non-null    float64
 5   glyhb      390 non-null    float64
 6   location   403 non-null    object 
 7   age        403 non-null    int64  
 8   gender     403 non-null    object 
 9   height     398 non-null    float64
 10  weight     402 non-null    float64
 11  frame      391 non-null    object 
 12  bp.1s      398 non-null    float64
 13  bp.1d      398 non-null    float64
 14  bp.2s      141 non-null    float64
 15  bp.2d      141 non-null    float64
 16  waist      401 non-null    float64
 17  hip        401 non-null    float64
 18  time.ppn   400 non-null    float64
 19  insurance  403 non-null    int64  
 20  fh        

In [5]:
df.describe(include='all')

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,insurance,fh,smoking,dm
count,403.0000,402.0000,403.0000,402.0000,402.0000,390.0000,403,403.0000,403,398.0000,402.0000,391,398.0000,398.0000,141.0000,141.0000,401.0000,401.0000,400.0000,403.0000,403.0000,403.0000,390
unique,nan,nan,nan,nan,nan,nan,2,nan,2,nan,nan,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2
top,nan,nan,nan,nan,nan,nan,Louisa,nan,female,nan,nan,medium,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,no
freq,nan,nan,nan,nan,nan,nan,203,nan,234,nan,nan,184,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,330
mean,15978.3102,207.8458,106.6725,50.4453,4.5216,5.5898,NaN,46.8511,NaN,66.0201,177.5920,NaN,136.9045,83.3216,152.3830,92.5248,37.9002,43.0399,341.2500,1.0546,0.1886,1.8462,NaN
std,11881.1221,44.4456,53.0767,17.2626,1.7279,2.2426,NaN,16.3123,NaN,3.9185,40.3407,NaN,22.7410,13.5892,21.7130,11.5552,5.7293,5.6567,309.5410,0.7961,0.3917,0.6512,NaN
min,1000.0000,78.0000,48.0000,12.0000,1.5000,2.6800,NaN,19.0000,NaN,52.0000,99.0000,NaN,90.0000,48.0000,110.0000,60.0000,26.0000,30.0000,5.0000,0.0000,0.0000,1.0000,NaN
25%,4792.5000,179.0000,81.0000,38.0000,3.2000,4.3800,NaN,34.0000,NaN,63.0000,151.0000,NaN,121.2500,75.0000,138.0000,84.0000,33.0000,39.0000,90.0000,0.0000,0.0000,1.0000,NaN
50%,15766.0000,204.0000,89.0000,46.0000,4.2000,4.8400,NaN,45.0000,NaN,66.0000,172.5000,NaN,136.0000,82.0000,149.0000,92.0000,37.0000,42.0000,240.0000,1.0000,0.0000,2.0000,NaN
75%,20336.0000,230.0000,106.0000,59.0000,5.4000,5.6000,NaN,60.0000,NaN,69.0000,200.0000,NaN,146.7500,90.0000,161.0000,100.0000,41.0000,46.0000,517.5000,2.0000,0.0000,2.0000,NaN


In [6]:
df.shape

(403, 23)

In [7]:
df.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age', 'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s', 'bp.2d', 'waist', 'hip', 'time.ppn', 'insurance', 'fh', 'smoking', 'dm'], dtype='object')

### Drop unwanted features

Using the same data set as before, now try another model with these predictor variables: age, cholesterol and insurance type. 

In [8]:
df2 = df[['age', 'chol', 'insurance','dm' ]]

In [9]:
df2.head()

,age,chol,insurance,dm
0,46,203.0000,1,no
1,29,165.0000,0,no
2,58,228.0000,2,no
3,67,78.0000,1,no
4,64,249.0000,0,yes


### Treat Missing Values

In [10]:
df2.isnull().sum()

age           0
chol          1
insurance     0
dm           13
dtype: int64

In [11]:
df2.dropna(inplace=True)

In [12]:
df2.isnull().sum()

age          0
chol         0
insurance    0
dm           0
dtype: int64

In [13]:
encoder = LabelEncoder()

In [14]:
df2["dm"] = encoder.fit_transform(df2["dm"])

In [15]:
df2["insurance"] = encoder.fit_transform(df2["insurance"].astype("str"))

In [16]:
df2

,age,chol,insurance,dm
0,46,203.0000,1,0
1,29,165.0000,0,0
2,58,228.0000,2,0
3,67,78.0000,1,0
4,64,249.0000,0,1
...,...,...,...,...
397,89,301.0000,1,0
398,53,296.0000,2,1
399,51,284.0000,2,0
400,29,194.0000,0,1


In [17]:
df2.insurance.value_counts()

1    144
2    132
0    113
Name: insurance, dtype: int64

In [18]:
df2["insurance"] = df2["insurance"].astype("object")

In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389 entries, 0 to 401
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        389 non-null    int64  
 1   chol       389 non-null    float64
 2   insurance  389 non-null    object 
 3   dm         389 non-null    int32  
dtypes: float64(1), int32(1), int64(1), object(1)
memory usage: 13.7+ KB


### Age Predictor

In [20]:
model_age = smf.logit(formula="dm ~ + age", data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.384479
         Iterations 7


In [21]:
model_age.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     dm   No. Observations:                  389
Model:                          Logit   Df Residuals:                      387
Method:                           MLE   Df Model:                            1
Date:                Fri, 07 May 2021   Pseudo R-squ.:                  0.1059
Time:                        15:37:56   Log-Likelihood:                -149.56
converged:                       True   LL-Null:                       -167.27
Covariance Type:            nonrobust   LLR p-value:                 2.667e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.3967      0.542     -8.107      0.000      -5.460      -3.334
age            0.0524      0.009      5.584      0.000       0.034       0.071
==============================================================================
"""

In [22]:
model_age.params

Intercept   -4.3967
age          0.0524
dtype: float64

In [23]:
np.exp(model_age.params)

Intercept   0.0123
age         1.0538
dtype: float64

Enter the odds ratio for age to two decimal places:

In [24]:
model_odds = pd.DataFrame(np.exp(model_age.params), columns=["ODDS RATIO"])

In [25]:
model_odds["Z-Value"] = model_age.pvalues

In [26]:
model_odds[["2.5%","97.5%"]] = np.exp(model_age.conf_int())

In [27]:
model_odds

,ODDS RATIO,Z-Value,2.5%,97.5%
Intercept,0.0123,0.0000,0.0043,0.0357
age,1.0538,0.0000,1.0346,1.0733


### Cholesterol Predictor

In [28]:
model_chol = smf.logit(formula="dm ~ + chol", data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.410820
         Iterations 6


In [29]:
model_chol.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     dm   No. Observations:                  389
Model:                          Logit   Df Residuals:                      387
Method:                           MLE   Df Model:                            1
Date:                Fri, 07 May 2021   Pseudo R-squ.:                 0.04460
Time:                        15:37:56   Log-Likelihood:                -159.81
converged:                       True   LL-Null:                       -167.27
Covariance Type:            nonrobust   LLR p-value:                 0.0001122
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.1700      0.683     -6.108      0.000      -5.508      -2.832
chol           0.0115      0.003      3.813      0.000       0.006       0.017
==============================================================================
"""

In [30]:
model_chol.params

Intercept   -4.1700
chol         0.0115
dtype: float64

In [31]:
np.exp(model_chol.params)

Intercept   0.0155
chol        1.0116
dtype: float64

Enter the odds ratio for cholesterol to two decimal places:

In [32]:
model_odds = pd.DataFrame(np.exp(model_chol.params), columns=["ODDS RATIO"])

In [33]:
model_odds["Z-Value"] = model_chol.pvalues

In [34]:
model_odds[["2.5%","97.5%"]] = np.exp(model_chol.conf_int())

In [35]:
model_odds

,ODDS RATIO,Z-Value,2.5%,97.5%
Intercept,0.0155,0.0000,0.0041,0.0589
chol,1.0116,0.0001,1.0056,1.0175


### Insurance Predictor

In [36]:
df2.head()

,age,chol,insurance,dm
0,46,203.0000,1,0
1,29,165.0000,0,0
2,58,228.0000,2,0
3,67,78.0000,1,0
4,64,249.0000,0,1


In [37]:
model_insurance = logit(formula="dm ~ C(insurance)", data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.427304
         Iterations 6


In [38]:
model_insurance.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     dm   No. Observations:                  389
Model:                          Logit   Df Residuals:                      386
Method:                           MLE   Df Model:                            2
Date:                Fri, 07 May 2021   Pseudo R-squ.:                0.006262
Time:                        15:37:56   Log-Likelihood:                -166.22
converged:                       True   LL-Null:                       -167.27
Covariance Type:            nonrobust   LLR p-value:                    0.3508
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.4198      0.238     -5.976      0.000      -1.885      -0.954
C(insurance)[T.1]    -0.3478      0.335     -1.038      0.299      -1.004       0.309
C(insurance)[T.2]    -0.4919      0.352     -1.397      0.162      -1.182       0.198
=====================================================================================
"""

In [39]:
model_odds = pd.DataFrame(np.exp(model_insurance.params), columns=["ODDS RATIO"])

In [40]:
model_odds

,ODDS RATIO
Intercept,0.2418
C(insurance)[T.1],0.7062
C(insurance)[T.2],0.6115


In [41]:
-----------------------

SyntaxError: invalid syntax (<ipython-input-41-c3524f4dc0d6>, line 1)

In [ ]:
govtinsurance = df2[df2["insurance"] == 1]

In [ ]:
govtinsurance

In [ ]:
----------------

In [ ]:
model_ins1 = smf.logit(formula="dm ~ + C(insurance)", data=govtinsurance).fit()

In [ ]:
model_ins1.summary()

In [ ]:
model_ins1.params

In [ ]:
np.exp(model_ins1.params)

Enter the odds ratio for insurance to two decimal places:

In [ ]:
model_odds = pd.DataFrame(np.exp(model_ins1.params), columns=["ODDS RATIO"])

In [ ]:
model_odds["Z-Value"] = model_ins1.pvalues

In [ ]:
model_odds[["2.5%","97.5%"]] = np.exp(model_ins1.conf_int())

In [ ]:
model_odds

In [ ]:
#dir(model_ins1)

In [ ]:
#model_ins.fittedvalues

In [ ]:
privinsurance = df2[df2["insurance"] == 2]

In [ ]:
privinsurance

In [ ]:
model_ins2 = smf.logit(formula="dm ~ + C(insurance)", data=privinsurance).fit()

In [ ]:
model_ins2.summary()

In [ ]:
model_ins2.params

In [ ]:
np.exp(model_ins2.params)

Enter the odds ratio for insurance to two decimal places:

In [ ]:
model_odds = pd.DataFrame(np.exp(model_ins2.params), columns=["ODDS RATIO"])

In [ ]:
model_odds["Z-Value"] = model_ins2.pvalues

In [ ]:
model_odds[["2.5%","97.5%"]] = np.exp(model_ins2.conf_int())

In [ ]:
model_odds

In [ ]:
df2

In [ ]:
df2.groupby('insurance')['dm'].count()

In [ ]:
govtinsurance.reset_index(inplace=True, drop=True)

In [ ]:
govtinsurance

In [ ]:
govtinsurance.groupby("dm").count()

In [ ]:
privinsurance.reset_index(inplace=True, drop=True)

In [ ]:
privinsurance

In [ ]:
privinsurance.groupby("dm").count()

#### Python code done by Dennis Lam